In [1]:
from pyspark.shell import spark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,BooleanType,DateType,BinaryType
from pyspark.sql.functions import spark_partition_id
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
from pyspark.sql.functions import md5

24/06/11 11:36:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/

Using Python version 3.8.10 (default, Nov 22 2023 10:22:35)
Spark context Web UI available at http://10.128.0.6:4040
Spark context available as 'sc' (master = yarn, app id = application_1718099923489_0004).
SparkSession available as 'spark'.


In [2]:
BUCKET_NAME = "mzinal-dproc1"
OUTPUT_PATH = "s3a://" + BUCKET_NAME + "/datagen1/"
numPart = 100
rowsPerPart = 500000 # 50Mb each file, 5G total

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [3]:
def genPartition(part):
  for row in part:
    for val in range(0, rowsPerPart):
      yield [val]

In [4]:
df1 = spark.createDataFrame(data=list(map(lambda x: [x], list(range(1, numPart+1)))), schema = ["id_part"])
df2 = df1.repartition(numPart).rdd.mapPartitions(genPartition).toDF(["val"])
df3 = df2.withColumn("val",df2.val.cast(IntegerType()))
df4 = df3.withColumn("spark_partition_id",spark_partition_id()) \
  .withColumn("int_1",col("val")+20) \
  .withColumn("int_2",col("val")+30) \
  .withColumn("int_3",col("val")+33) \
  .withColumn("int_4",col("val")+37) \
  .withColumn("int_5",col("val")+1000000000) \
  .withColumn("int_6",col("val")+2000000000) \
  .withColumn("long_1",col("val")+3000000000) \
  .withColumn("long_2",col("val")+4100000000) \
  .withColumn("long_3",col("val")+4200000000) \
  .withColumn("long_4",col("val")+4300000000) \
  .withColumn("long_5",col("val")+4400000000) \
  .withColumn("long_6",col("val")+4500000000) \
  .withColumn("double_1", expr("CAST(int_6/8 AS double)")) \
  .withColumn("double_2", expr("CAST(int_6/10 AS double)")) \
  .withColumn("decimal_1", expr("CAST(int_6/100 AS DECIMAL(10,2))")) \
  .withColumn("decimal_2", expr("CAST(int_3/100 AS DECIMAL(10,2))")) \
  .withColumn("decimal_3", expr("CAST(int_2/100 AS DECIMAL(10,2))")) \
  .withColumn("str_1",md5(col("int_1").cast(BinaryType()))) \
  .withColumn("str_2",col("str_1")) \
  .withColumn("str_3",col("str_1")) \
  .withColumn("str_4",col("str_1"))

In [5]:
df4.write.mode("overwrite").parquet(OUTPUT_PATH)

In [6]:
df = spark.read.parquet(OUTPUT_PATH)
df.printSchema
df.createOrReplaceTempView("temp1")

In [7]:
spark.sql("SELECT * FROM temp1 LIMIT 20")

val,spark_partition_id,int_1,int_2,int_3,int_4,int_5,int_6,long_1,long_2,long_3,long_4,long_5,long_6,double_1,double_2,decimal_1,decimal_2,decimal_3,str_1,str_2,str_3,str_4
0,5,20,30,33,37,1000000000,2000000000,3000000000,4100000000,4200000000,4300000000,4400000000,4500000000,2.5E8,2.0E8,20000000.00,0.33,0.30,8e97382f2e6a1fe66...,8e97382f2e6a1fe66...,8e97382f2e6a1fe66...,8e97382f2e6a1fe66...
1,5,21,31,34,38,1000000001,2000000001,3000000001,4100000001,4200000001,4300000001,4400000001,4500000001,2.50000000125E8,2.000000001E8,20000000.01,0.34,0.31,009d30980aa408f62...,009d30980aa408f62...,009d30980aa408f62...,009d30980aa408f62...
2,5,22,32,35,39,1000000002,2000000002,3000000002,4100000002,4200000002,4300000002,4400000002,4500000002,2.5000000025E8,2.000000002E8,20000000.02,0.35,0.32,0764924ee9c3cff6d...,0764924ee9c3cff6d...,0764924ee9c3cff6d...,0764924ee9c3cff6d...
3,5,23,33,36,40,1000000003,2000000003,3000000003,4100000003,4200000003,4300000003,4400000003,4500000003,2.50000000375E8,2.000000003E8,20000000.03,0.36,0.33,4dd9845b20c0205b6...,4dd9845b20c0205b6...,4dd9845b20c0205b6...,4dd9845b20c0205b6...
4,5,24,34,37,41,1000000004,2000000004,3000000004,4100000004,4200000004,4300000004,4400000004,4500000004,2.500000005E8,2.000000004E8,20000000.04,0.37,0.34,b4526e59a534303ef...,b4526e59a534303ef...,b4526e59a534303ef...,b4526e59a534303ef...
5,5,25,35,38,42,1000000005,2000000005,3000000005,4100000005,4200000005,4300000005,4400000005,4500000005,2.50000000625E8,2.000000005E8,20000000.05,0.38,0.35,59dea06d5a405f950...,59dea06d5a405f950...,59dea06d5a405f950...,59dea06d5a405f950...
6,5,26,36,39,43,1000000006,2000000006,3000000006,4100000006,4200000006,4300000006,4400000006,4500000006,2.5000000075E8,2.000000006E8,20000000.06,0.39,0.36,7debb56a86892babf...,7debb56a86892babf...,7debb56a86892babf...,7debb56a86892babf...
7,5,27,37,40,44,1000000007,2000000007,3000000007,4100000007,4200000007,4300000007,4400000007,4500000007,2.50000000875E8,2.000000007E8,20000000.07,0.40,0.37,1a6f00ceda41c4f90...,1a6f00ceda41c4f90...,1a6f00ceda41c4f90...,1a6f00ceda41c4f90...
8,5,28,38,41,45,1000000008,2000000008,3000000008,4100000008,4200000008,4300000008,4400000008,4500000008,2.50000001E8,2.000000008E8,20000000.08,0.41,0.38,a54666d8d67223511...,a54666d8d67223511...,a54666d8d67223511...,a54666d8d67223511...
9,5,29,39,42,46,1000000009,2000000009,3000000009,4100000009,4200000009,4300000009,4400000009,4500000009,2.50000001125E8,2.000000009E8,20000000.09,0.42,0.39,e3a98ec981a4c757b...,e3a98ec981a4c757b...,e3a98ec981a4c757b...,e3a98ec981a4c757b...
